## This notebook was only created to test the ReadAndRunAllWorkFlows.py


<br>
<br>

In [1]:
#pull the ara results and look through KF+G edge graphs and pull the povenance attribute and make a table that will look like
#rows will be kp
#col be workflows
#entry would be the ara's that had edges in kp in the workflow. 

In [2]:
import requests
import json
from collections import defaultdict
import os
import pandas as pd


#https://ars.ci.transltr.io/ars/api --- 
#https://arax.ci.transltr.io -- backup url

In [3]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

In [4]:
def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results'
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error'
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error'
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' 
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return dictionary


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))

<br>

### Submiting more than one jobs (test)

<br>

In [ ]:
urls = ['/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.9_EGFR_advanced.json', '/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.1_RHOBTB2.json']

In [ ]:
dict_workflows = {}
for i in urls:
    feature = (os.path.splitext(os.path.basename(i))[0])
    print(i, feature)
    with open(i,'r') as inf:
        query = json.load(inf)
        kcresult = submit_to_ars(query)
        #xx = retrieve_devars_results(kcresult)
        
        dict_workflows[feature] = kcresult

In [ ]:
dict_workflows

In [ ]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_ars_results(val)
    
    workflow_result_messages[keys] = result_status

In [ ]:
workflow_result_messages

In [ ]:
col = []
final_dict = defaultdict(list)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k].items():
        if key.startswith('kp-'):
            key_mod = key.replace('kp-','')
        else:
            key_mod = key
        
        final_dict[key_mod].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

In [ ]:
df

In [ ]:
df.loc['pk_id']

In [ ]:
df.to_excel('test_ara_worklow.xlsx')

In [ ]:
pwd

<br>

## Test for individual run

<br>

In [5]:
with open('/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowD/D.1_parkinsons-crohns.json') as inf:
    query1 = json.load(inf)

In [6]:
printjson(query1)

{
    "message": {
        "query_graph": {
            "edges": {
                "e00": {
                    "subject": "n00",
                    "object": "n01"
                },
                "e01": {
                    "subject": "n01",
                    "object": "n02"
                }
            },
            "nodes": {
                "n00": {
                    "ids": [
                        "DOID:14330"
                    ],
                    "categories": [
                        "biolink:Disease"
                    ]
                },
                "n01": {
                    "categories": [
                        "biolink:NamedThing"
                    ]
                },
                "n02": {
                    "ids": [
                        "DOID:8778"
                    ],
                    "categories": [
                        "biolink:Disease"
                    ]
                }
            }
        }
    }
}


In [7]:
kcresult = submit_to_ars(query1)

https://arax.ncats.io/?source=ARS&id=e90c4df8-2fa6-4459-9409-013fcb197a52


In [10]:
retrieve_ars_results(kcresult)

Running


{}

In [ ]:
kcresult

In [11]:
kcresult = 'eb183cbc-9ea6-4899-a34b-1ec508b38c43'
ars_url='https://ars.transltr.io/ars/api'
pk = 'https://arax.ncats.io/?source=ARS&id=' + kcresult
message_url = f'{ars_url}/messages/{kcresult}?trace=y'
response = requests.get(message_url)
j = response.json()

In [12]:
j.keys()

dict_keys(['message', 'status', 'actor', 'children'])

In [13]:
j

{'message': 'eb183cbc-9ea6-4899-a34b-1ec508b38c43',
 'status': 'Done',
 'actor': {'pk': 9,
  'channel': 'general',
  'agent': 'ars-default-agent',
  'path': ''},
 'children': [{'message': 'c9f7f1d7-6067-46c0-8484-1a2e4678bc28',
   'status': 'Done',
   'code': 200,
   'actor': {'pk': 1,
    'channel': 'general',
    'agent': 'ara-aragorn',
    'path': 'runquery'},
   'children': []},
  {'message': '84ee69dc-0ea4-4d3d-9f35-914bc05d40fc',
   'status': 'Done',
   'code': 200,
   'actor': {'pk': 2,
    'channel': 'general',
    'agent': 'ara-arax',
    'path': 'runquery'},
   'children': []},
  {'message': '56fd97f0-195f-4bee-8301-dac00acf0773',
   'status': 'Error',
   'code': 598,
   'actor': {'pk': 3,
    'channel': 'general',
    'agent': 'ara-bte',
    'path': 'runquery'},
   'children': []},
  {'message': '4b9ddd13-e634-44c3-8639-a6f121b0bc27',
   'status': 'Error',
   'code': 504,
   'actor': {'pk': 6,
    'channel': 'general',
    'agent': 'ara-unsecret',
    'path': 'runquery'},
  

In [145]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [150]:
dictionary_test = {}
for child in j['children']:
    print(child['actor']['agent'])
    dictionary_test
    if child['status']  == 'Done':
        childmessage_id = child['message']
        child_url = f'{ars_url}/messages/{childmessage_id}'
        child_response = requests.get(child_url).json()
        #print(child_response['fields']['data']['message']['knowledge_graph']['edges'])
        if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
            test_att_values =[]
            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                #print(val)
                
                att = list(edge_ex[val]['attributes'])
                #print(att)
                for tx in att:
                    #print(tx)
                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source'):
                        #print(tx['value'])
                        x = tx['value']
                        
                        test_att_values.append(x)
                        test_att = set(flatten_list(test_att_values))
                        
                        print(test_att_values)
                        dictionary_test[child['actor']['agent']] = test_att
        else:
            pass
    

ara-aragorn


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




['infores:rtx-kg2', 'infores:semmeddb', 'infores:aragorn', 'infores:rtx-kg2', 'infores:semmeddb', 'infores:aragorn', 'infores:rtx-kg2', 'infores:semmeddb', 'infores:aragorn', 'infores:rtx-kg2', 'infores:semmeddb', 'infores:aragorn', 'infores:rtx-kg2', 'infores:semmeddb', 'infores:aragorn', 'infores:rtx-kg2', 'infores:semmeddb', 'infores:aragorn', 'infores:rtx-kg2', 'infores:semmeddb', 'infores:aragorn', 'infores:rtx-kg2', 'infores:semmeddb', 'infores:aragorn', 'infores:rtx-kg2', 'infores:semmeddb', 'infores:aragorn', 'infores:rtx-kg2', 'infores:disgenet', 'infores:aragorn', 'infores:rtx-kg2', 'infores:semmeddb', 'infores:aragorn', 'infores:rtx-kg2', 'infores:disgenet', 'infores:aragorn', 'infores:rtx-kg2', 'infores:semmeddb', 'infores:aragorn', 'infores:rtx-kg2', 'infores:semmeddb', 'infores:aragorn', 'infores:rtx-kg2', 'infores:aragorn', 'infores:rtx-kg2', 'infores:semmeddb', 'infores:aragorn', 'infores:rtx-kg2', 'infores:aragorn', 'infores:rtx-kg2', 'infores:aragorn', 'infores:rtx-k

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



ara-improving
['infores:gwas-catalog']
['infores:gwas-catalog', 'infores:spoke']
['infores:gwas-catalog', 'infores:spoke', 'infores:improving-agent']
['infores:gwas-catalog', 'infores:spoke', 'infores:improving-agent', 'infores:diseases']
['infores:gwas-catalog', 'infores:spoke', 'infores:improving-agent', 'infores:diseases', 'infores:spoke']
['infores:gwas-catalog', 'infores:spoke', 'infores:improving-agent', 'infores:diseases', 'infores:spoke', 'infores:improving-agent']
['infores:gwas-catalog', 'infores:spoke', 'infores:improving-agent', 'infores:diseases', 'infores:spoke', 'infores:improving-agent', 'infores:gwas-catalog']
['infores:gwas-catalog', 'infores:spoke', 'infores:improving-agent', 'infores:diseases', 'infores:spoke', 'infores:improving-agent', 'infores:gwas-catalog', 'infores:spoke']
['infores:gwas-catalog', 'infores:spoke', 'infores:improving-agent', 'infores:diseases', 'infores:spoke', 'infores:improving-agent', 'infores:gwas-catalog', 'infores:spoke', 'infores:improvin

ara-aragorn-exp
kp-openpredict
kp-icees
ara-robokop
kp-chp
kp-cohd
kp-icees-dili


In [149]:
dictionary_test

{'ara-aragorn': {'CTD',
  'DisGeNET',
  'JensenLab Knowledge GHR',
  'JensenLab Knowledge UniProtKB-KW',
  'JensenLab Text Mining',
  'Monarch',
  'eRAM',
  'infores:aragorn',
  'infores:aragorn-ranker-ara',
  'infores:automat-robokop',
  'infores:automat.cord19',
  'infores:automat.covidkop',
  'infores:automat.hetio',
  'infores:automat.mychem-info',
  'infores:automat.ontology-hierarchy',
  'infores:automat.pharos',
  'infores:automat.robokop',
  'infores:automat.uberongraph',
  'infores:automat:biolink',
  'infores:chembl',
  'infores:clinvar',
  'infores:cord19-scibite',
  'infores:diseases',
  'infores:disgenet',
  'infores:gencc',
  'infores:genetics-data-provider',
  'infores:gwas-catalog',
  'infores:hetio',
  'infores:mesh',
  'infores:mondo',
  'infores:ontological-hierarchy',
  'infores:pharos',
  'infores:rtx-kg2',
  'infores:semmeddb',
  'infores:spoke',
  'infores:sri-reference-kg',
  'infores:ubergraph'},
 'ara-arax': {'infores:arax',
  'infores:chembl',
  'infores:disg

In [ ]:
dictionary={}
results = {}
dictionary_2 = {}
for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                print(val)
                                test_att_values =[]
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source'):
                                        test_att_values.append(tx['value'])
                                        dictionary_2[child['actor']['agent']] = test_att_values
                    else:
                        pass 
                else:
                    dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results'
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error'
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error'
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' 
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)

In [ ]:
dictionary_2

In [ ]:
for child in j['children']:
    #print(child['status'])
    if child['status']  == 'Done':
        childmessage_id = child['message']
        child_url = f'{ars_url}/messages/{childmessage_id}'
        child_response = requests.get(child_url).json()
        if child_response['fields']['data']['message']['knowledge_graph']['edges']:
            print('correct')
        else:
            print('incorrect')

In [ ]:
child_response['fields']['data']['message']['knowledge_graph']

In [ ]:
mm = child_response['pk']
mm

In [ ]:
xt = f'{ars_url}/messages/{childmessage_id}/{mm}'
xt

In [ ]:
f'{ars_url}/model/{mm}'

In [ ]:
requests.get(xt).json

In [ ]:
childmessage_id = child['message']
child_url = f'{ars_url}/messages/{childmessage_id}'

 child_response = requests.get(child_url).json()

In [ ]:
result_status = retrieve_ars_results(kcresult)

In [ ]:
kcresult

In [ ]:
result_status 

In [ ]:
pwd

In [ ]:
df

In [ ]:
def make_hyperlink(value):
    #url = "https://custom.url/{}"
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

In [ ]:
make_hyperlink('https://arax.ncats.io/?source=ARS&id=325cfd91-ee6c-48eb-8a09-7226bbafb482')

In [ ]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [ ]:
df

In [ ]:
df.to_excel('test_ara_worklow.xlsx')

In [ ]:
df